## 1. Importing Libraries

In [28]:
# install the tranformer libraries using bleow command

# ! pip install transformers

In [29]:
# importing libraries

from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [30]:
# input sentence

input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"

In [31]:
# loading the large gpt tokenizer & gpt model

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id = tokenizer.eos_token_id)

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
#GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

#view model parameters
GPT2.summary()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  774030080 
 er)                                                             
                                                                 
Total params: 774030080 (2.88 GB)
Trainable params: 774030080 (2.88 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
#for reproducability
SEED = 34

#maximum number of words in output text
MAX_LEN = 70

In [33]:
#get deep learning basics
import tensorflow as tf
tf.random.set_seed(SEED)

### First Pass (Greedy Search)

In [34]:
# Encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors = 'tf')

# Generate the text until the output length(which include context length) reaches 50
greedy_output = GPT2.generate(input_ids, max_length = MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work: go to the gym.

I'm not talking about the gym that's right next to my house. I'm talking about the gym that's right next to my office.

I'm not talking about the gym that


### Beam Search with N-Gram Penalities

In [35]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids,
    max_length = MAX_LEN,
    num_beams = 5,
    no_repeat_ngram_size = 2,
    num_return_sequences = 5,
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's not a good movie. I mean, it's
1: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a girl
2: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's about a guy who has a crush on a woman
3: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," 

### Basic Sampling

In [36]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
                             input_ids,
                             do_sample = True,
                             max_length = MAX_LEN,
                             top_k = 0,
                             temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work: drink some beer. Ask anyone who says otherwise.

If you're like me and you work in an office with windows that can't possibly be broken, I can assure you that you're not alone. The truth is,


### top-k-sampling

In [37]:
#sample from only top_k most likely words
sample_output = GPT2.generate(
                             input_ids,
                             do_sample = True,
                             max_length = MAX_LEN,
                             top_k = 50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work or school: take a stroll and catch up on my mail. This is not something that can be done in the privacy of my office during normal business hours. My office is right here, and there are a lot of people who would ...


### Top-p-sampling

In [38]:
#sample only from 80% most likely words
sample_output = GPT2.generate(
                             input_ids,
                             do_sample = True,
                             max_length = MAX_LEN,
                             top_p = 0.8,
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I don't know about you, but there's only one thing I want to do after a long day of work, and that's play a guitar.

Related Content:

No, It's Not All About the Pitch in Kevin Drew's No, It's Not All About the Pitch

John Lennon: "It's Not About ...


### Top-k-sampling & top-p-sampling combining both approachs

In [39]:
#combine both sampling techniques
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = 2*MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .7,
                              top_k = 50,
                              top_p = 0.85,
                              num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: I don't know about you, but there's only one thing I want to do after a long day of work… and that's to take a break from the computer. Sure, I'll still use my tablet as well, but the keyboard is so nice that it's almost worth it. The keyboard on my X99 Pro is just so nice to type on. And even if I did get rid of the tablet, I'd still use it to keep track of what's going on in my system.

The other thing I like about the keyboard on my X99 Pro is that the layout is super-easy to type on. The keys are so spaced that you'll be able to...

1: I don't know about you, but there's only one thing I want to do after a long day of work: sit down and play some videogames with some friends. But if there's one thing I can say about my friend that's true, it's that we've all been together a long time, and we're still friends. But it's hard to stay friends with the same person for a whole 

## III. Benchmark Prompts

In [40]:
MAX_LEN = 150

### In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

In [41]:

prompt1 = 'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.'

input_ids = tokenizer.encode(prompt1, return_tensors='tf')

In [42]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The researchers, led by biologist and professor of linguistics at Uppsala University, Anna-Kaisa Häggström, studied a group of wild unicorns called Gälltli, who are native to Argentina and Brazil. Their territory stretches from the foothills of the Andes Mountains in Peru and Argentina all the way up to the Amazon basin in Brazil.

While most unicorns live in the mountains, the researchers found that the animals live in a valley near the town of Ch...



### Giant Robot Found Beneath Metropolis, Sparks Debate Among Scientists.

In [49]:
# Can we use GPT-2 to generate fake news stories?
prompt2 = 'Giant Robot Found Beneath Metropolis, Sparks Debate Among Scientists'

input_ids = tokenizer.encode(prompt2, return_tensors='tf')

In [50]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Giant Robot Found Beneath Metropolis, Sparks Debate Among Scientists (VIDEO)

The "Giant Robot" video is a spoof video created by the makers of "Sharknado" with a humorous twist.

The video stars the famous actor Will Ferrell and features the voice of actor Stephen Baldwin, and features the famous phrase "Giant Robot" repeated multiple times.

The clip, which is being shared online by a number of entertainment websites, was produced by the "Sharknado" filmmakers and features Ferrell as a scientist, and Baldwin as a scientist.

The clip is being shared on Facebook and other social media sites, with many people praising the film and questioning whether the actors really are scientists...



### Was Karna’s rivalry with Arjuna inevitable in Mahabharata?

In [45]:
# Can we use GPT-2 to imagine alternate histories of Lord of the Rings?

prompt3 = 'Was Karna’s rivalry with Arjuna inevitable in Mahabharata?'

input_ids = tokenizer.encode(prompt3, return_tensors='tf')

In [46]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Was Karna’s rivalry with Arjuna inevitable in Mahabharata?

It is true that Karna was the first king of the Pandava kingdom and later on he was the head of the clan of Pandava kings in Arjuna's army. But this is not the first time that Arjuna and his army encountered Karna. In the Mahabharata, Karna's battle with Arjuna's army and its victory was already pre-planned. Karna's role was that of the spear-carrier who is in the front line, and Arjuna's role was that of the general who was to spearhead the attack.

In this respect, Karna is a very modern...



### For today’s homework assignment, please describe the reasons for the 1991 kargil war in india.



In [47]:
# Can we use GPT-2 to do our homework?

prompt4 = "For today’s homework assignment, please describe the reasons for the 1991 kargil war in india."

input_ids = tokenizer.encode(prompt4, return_tensors='tf')

In [48]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True,
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50,
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: For today’s homework assignment, please describe the reasons for the 1991 kargil war in india.

It is quite simple. We had been at war with Pakistan for over 20 years. In that time, over 2,000 people have lost their lives.

This is the main reason why India had been hesitant in the past.

We are still at war with Pakistan. This is also a reason why India needs to build a strong India-Pakistan alliance.

As part of that, the country must take a stand on Kashmir, the problem that has created tension and discord between India and Pakistan.

In the past, we have supported Kashmiri separatists. Now we must support those who want peace....

